Notebook to create MySQL database from .csv file

In [16]:
import pandas as pd 
import mysql.connector as mysql
from mysql.connector import Error

path = 'data/data_lyrics_features.csv.zip'
data = pd.read_csv(path, index_col = 0)

path = 'data/id_lookup.csv.zip'
lookup_table = pd.read_csv(path, index_col=0)

In [11]:
data.head()

,id,Lyrics,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,2ne4ZuEhn35sGRftyI8jJP,"No, no\r\nI ain't ever trapped out the bando\r...",0.783,0.563,-8.613,0.2330,0.370000,0.000000,0.1090,0.634,114.897
1,6lHGPW1IW2w5rQ3FFqqmkx,"The drinks go down and smoke goes up, I feel m...",0.781,0.571,-10.630,0.0723,0.065700,0.175000,0.1140,0.631,124.021
2,6QHWyeh90gahr79P36m7gQ,She don't live on planet Earth no more\r\nShe ...,0.705,0.764,-7.407,0.0862,0.016800,0.000036,0.1340,0.240,159.995
3,3mBEewFYvVpVIVAFnuECyR,"Trippin' off that Grigio, mobbin', lights low\...",0.404,0.523,-7.099,0.0485,0.644000,0.000004,0.1330,0.537,134.385
4,4PoGx0KVpaqVWxSF9j11O3,"I see a midnight panther, so gallant and so br...",0.293,0.604,-8.742,0.0426,0.000607,0.000015,0.0877,0.375,79.946


Connect to MySQL and create database

In [12]:
try:
    conn = mysql.connect(host='localhost', 
                        user='robin',  
                        password='sqlpassword')
    
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("CREATE DATABASE data_musiccbr")
        print("Database is created")
        
except Error as e:
    print("Error while connecting to MySQL", e)

Error while connecting to MySQL 1007 (HY000): Can't create database 'data_musiccbr'; database exists


Create song_data table and add data from csv file

In [13]:
try:
    conn = mysql.connect(host='localhost', database='data_musiccbr', user='robin', password='sqlpassword')
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("You're connected to database: ", record)
        cursor.execute('DROP TABLE IF EXISTS song_data;')
        cursor.execute("CREATE TABLE song_data(id CHAR(22), lyrics LONGTEXT, danceability FLOAT(10), energy FLOAT(10), loudness FLOAT(10), speechiness FLOAT(10), acousticness FLOAT(10), intrumentalness FLOAT(10), liveness FLOAT(10), valence FLOAT(10), tempo FLOAT(10))")
        print("Table is created....")
        #loop through the data frame
        for i,row in data.iterrows():
            sql = "INSERT INTO data_musiccbr.song_data VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
            cursor.execute(sql, tuple(row))
            conn.commit()
        
except Error as e:
            print("Error while connecting to MySQL", e)

You're connected to database:  ('data_musiccbr',)
Table is created....


Create lookup_table and add data from csv file

In [17]:
lookup_table.head()

,Song,Band,id
0,Everyday,Elijah Blake,2ne4ZuEhn35sGRftyI8jJP
1,Live Till We Die,Elijah Blake,6lHGPW1IW2w5rQ3FFqqmkx
2,The Otherside,Elijah Blake,6QHWyeh90gahr79P36m7gQ
3,Pinot,Elijah Blake,3mBEewFYvVpVIVAFnuECyR
4,Shadows & Diamonds,Elijah Blake,4PoGx0KVpaqVWxSF9j11O3


In [18]:
try:
    conn = mysql.connect(host='localhost', database='data_musiccbr', user='robin', password='sqlpassword')
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("You're connected to database: ", record)
        cursor.execute('DROP TABLE IF EXISTS lookup_table;')
        cursor.execute("CREATE TABLE lookup_table(Song TEXT(10000), Band TEXT(10000), id CHAR(22))")
        print("Table is created....")
        #loop through the data frame
        for i,row in lookup_table.iterrows():
            sql = "INSERT INTO data_musiccbr.lookup_table VALUES (%s,%s,%s)"
            cursor.execute(sql, tuple(row))
            conn.commit()
        
except Error as e:
            print("Error while connecting to MySQL", e)

You're connected to database:  ('data_musiccbr',)
Table is created....


In [27]:
import streamlit as st

try:
    conn = mysql.connect(**st.secrets["mysql"])
    if conn.is_connected():
        print("Connected")
    data = pd.read_sql('SELECT * FROM song_data', conn)

except Error as e:
    print("Error while connecting to MySQL", e)

Connected


In [24]:
data

,id,lyrics,danceability,energy,loudness,speechiness,acousticness,intrumentalness,liveness,valence,tempo
0,2ne4ZuEhn35sGRftyI8jJP,"No, no\r\nI ain't ever trapped out the bando\r...",0.783,0.563,-8.613,0.2330,0.370000,0.000000,0.1090,0.6340,114.897
1,6lHGPW1IW2w5rQ3FFqqmkx,"The drinks go down and smoke goes up, I feel m...",0.781,0.571,-10.630,0.0723,0.065700,0.175000,0.1140,0.6310,124.021
2,6QHWyeh90gahr79P36m7gQ,She don't live on planet Earth no more\r\nShe ...,0.705,0.764,-7.407,0.0862,0.016800,0.000036,0.1340,0.2400,159.995
3,3mBEewFYvVpVIVAFnuECyR,"Trippin' off that Grigio, mobbin', lights low\...",0.404,0.523,-7.099,0.0485,0.644000,0.000004,0.1330,0.5370,134.385
4,4PoGx0KVpaqVWxSF9j11O3,"I see a midnight panther, so gallant and so br...",0.293,0.604,-8.742,0.0426,0.000607,0.000015,0.0877,0.3750,79.946
...,...,...,...,...,...,...,...,...,...,...,...
348538,1RMFkQFXZU8Dqb6GWxb92M,Facing the storm\r\nAnd there's no one by my s...,0.342,0.852,-6.477,0.0747,0.001380,0.596000,0.0869,0.2190,134.515
348539,19zjNdVlA7hYlm1nF3xhyM,The ocean's beauty\r\nNo longer moves my heart...,0.276,0.930,-7.234,0.1050,0.000055,0.496000,0.1300,0.0544,93.967
348540,3cBMgg4VYGlixkPcHhIisM,Show me your banner\r\nCome show me your sign\...,0.615,0.950,-5.966,0.0665,0.002590,0.272000,0.0383,0.3950,95.015
348541,0U9taqEwyAFOK2Eu6riDtS,Amuse me\r\nSweet son of love\r\nSweet son of ...,0.326,0.951,-7.161,0.1940,0.002280,0.334000,0.0761,0.0488,93.211
